In [2]:
import seaborn as sns
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
os.chdir("/Users/zyxu/Documents/py/FM_weights")
from src.model import GPT

torch.manual_seed(1234)
model = GPT.from_pretrained("gpt2")
L = model.config.n_layer
H = model.config.n_head
C = model.config.n_embd

sigma_multiplier = np.sqrt(2 * np.log(C**2))  # denoising parameter
top_num = 60  # how many top indices to show


def my_round(a, fac=1e3):
    return np.round(a * fac) / fac

/Users/zyxu/opt/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
L = model.config.n_layer
H = model.config.n_head
C = model.config.n_embd
print(L,H,C)

12 12 768


In [3]:
block = model.transformer.h[0]
block

Block(
  (ln_1): LayerNorm()
  (attn): CausalSelfAttention(
    (c_attn): Linear(in_features=768, out_features=2304, bias=True)
    (c_proj): Linear(in_features=768, out_features=768, bias=True)
    (attn_dropout): Dropout(p=0.0, inplace=False)
    (resid_dropout): Dropout(p=0.0, inplace=False)
  )
  (ln_2): LayerNorm()
  (mlp): MLP(
    (c_fc): Linear(in_features=768, out_features=3072, bias=True)
    (gelu): GELU(approximate='none')
    (c_proj): Linear(in_features=3072, out_features=768, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
)

In [6]:
W_q, W_k, _ = block.attn.c_attn.weight.split(C, dim=0)
print(W_q.shape, W_k.shape)

torch.Size([768, 768]) torch.Size([768, 768])


In [7]:
W_q = W_q.T.view(C, H, C // H)
W_k = W_k.T.view(C, H, C // H)

In [8]:
print(W_q.shape, W_k.shape)

torch.Size([768, 12, 64]) torch.Size([768, 12, 64])
